# Probability of Success

The probability of measuring an ancilla qubit as 0 after each time evolution is given by:

$$\text{P(N, N}_{{\text{super}}}) = \left[\frac{1}{\sigma\sqrt{2\pi}} \int_{-\infty}^{\infty} e^{\frac{t^2}{2\sigma^2}} \prod_{k=0}^{N_{super}-1} \text{cos}^2\left(\frac{\Delta Et}{2^{k+1}}\right)dt\ \right]^N$$

Although we cannot directly calculate the width

In [2]:
import numpy as np
1 / (32 * np.exp(30625 / 2)) * (1 + 2 * np.exp(8125 / 2) + 3 * np.exp(7500) + 4 * np.exp(20625 / 2) + 5 * np.exp(12500) + 6 * np.exp(28125 / 2) + 7 * np.exp(15000) + 4 * np.exp(30625 / 2))

C:\Users\sirac\AppData\Local\Temp\ipykernel_5176\1467082041.py:2: RuntimeWarning: overflow encountered in exp
  1 / (32 * np.exp(30625 / 2)) * (1 + 2 * np.exp(8125 / 2) + 3 * np.exp(7500) + 4 * np.exp(20625 / 2) + 5 * np.exp(12500) + 6 * np.exp(28125 / 2) + 7 * np.exp(15000) + 4 * np.exp(30625 / 2))
C:\Users\sirac\AppData\Local\Temp\ipykernel_5176\1467082041.py:2: RuntimeWarning: invalid value encountered in scalar multiply
  1 / (32 * np.exp(30625 / 2)) * (1 + 2 * np.exp(8125 / 2) + 3 * np.exp(7500) + 4 * np.exp(20625 / 2) + 5 * np.exp(12500) + 6 * np.exp(28125 / 2) + 7 * np.exp(15000) + 4 * np.exp(30625 / 2))


nan

In [ ]:
np.exp